In [9]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/x_test.csv")


display(x_train.head())
display(y_train.head())
display(target.head())

train = pd.merge(x_train, y_train, on = 'ID')
target = x_test.copy()

,ID,model,year,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,0,I30,2019,Manual,21,Petrol,150,34.0,2.0
1,1,Santa Fe,2018,Semi-Auto,10500,Diesel,145,39.8,2.2
2,2,Tucson,2017,Manual,29968,Diesel,30,61.7,1.7
3,3,Kona,2018,Manual,27317,Petrol,145,52.3,1.0
4,4,Tucson,2018,Semi-Auto,31459,Diesel,145,57.7,1.7


,ID,price
0,0,23995
1,1,28490
2,2,13251
3,3,14990
4,4,17591


,ID,model,year,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,21,I10,2018,Manual,7374,Petrol,145,57.7,1.2
1,28,I30,2011,Automatic,59220,Diesel,160,49.6,1.6
2,33,Santa Fe,2019,Automatic,100,Diesel,145,39.2,2.2
3,38,Tucson,2016,Semi-Auto,49000,Diesel,125,57.6,1.7
4,45,I20,2018,Manual,30012,Petrol,145,55.4,0.0


In [36]:
# print(train.shape)
# print()
# print(train.info())
# print()
# print(train.describe())
# print()
# print(train.isnull().sum())
# print(target.shape)
# print()
# print(target.info())
# print()
# print(target.describe())
# print()
# print(target.isnull().sum())
x = train.drop(['ID','price'], axis=1)
y = train['price']
target_x = target.drop('ID', axis=1)

from sklearn.preprocessing import StandardScaler
ss_col = x.select_dtypes(exclude='object').columns
ss = StandardScaler()
x[ss_col] = ss.fit_transform(x[ss_col])
target_x[ss_col] = ss.transform(target_x[ss_col])

from sklearn.preprocessing import LabelEncoder
obj_col = train.select_dtypes(include='object').columns
# for col in obj_col:
#   print(col)
#   print(train[col].unique())
#   print(target_x[col].unique())

df = pd.concat([x, target_x])
le_col = ['model','year']
ohe_col = ['fuelType','transmission']

for col in le_col:
  le = LabelEncoder()
  df[col] = le.fit_transform(df[col])

df = pd.get_dummies(df, columns= ohe_col, dtype=int)

x_pre = df[:len(x)]
target_x_pre = df[len(x):]
print(x_pre.shape)
print(x.shape)
print(target_x.shape)
print(target_x_pre.shape)

from sklearn.model_selection import train_test_split
rs =810
x_train, x_val, y_train, y_val = train_test_split(x_pre, y, test_size=0.2, random_state=rs)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=rs)

from sklearn.model_selection import GridSearchCV

param = {'n_estimators':range(3,15),
         'max_depth':range(3,15)}
gscv = GridSearchCV(estimator=rf, param_grid=param, cv=5)
gscv.fit(x_train, y_train)
print(gscv.best_params_)
model = gscv.best_estimator_

pred_val = model.predict(x_val)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print('mae:', mean_absolute_error(y_val, pred_val))
print('mse:', mean_squared_error(y_val, pred_val))
print('rmse:', mean_squared_error(y_val, pred_val, squared=False))
print('rmse2:', mean_squared_error(y_val,pred_val)**0.5)
print('r2_score:', r2_score(y_val,pred_val))

pred = model.predict(target_x_pre)
result = pd.DataFrame({'ID':target['ID'], 'pred':pred})
result.to_csv('result.csv', index=False)
print(result)

(3888, 14)
(3888, 8)
(972, 8)
(972, 14)
{'max_depth': 9, 'n_estimators': 14}
mae: 822.1489514019993
mse: 1418198.9016497277
rmse: 1190.8815649130386
rmse2: 1190.8815649130386
r2_score: 0.9546563566243379
       ID          pred
0      21   8215.666716
1      28   4356.821650
2      33  35244.635949
3      38  13968.251786
4      45   8460.978315
..    ...           ...
967  4833   2054.813242
968  4840  19199.112181
969  4842   4561.607143
970  4843   6222.428571
971  4848   7814.628918

[972 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [35]:
print(y.std())

5878.240687326913


In [37]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv")
df.head()

,time,이화동강수,이화동기온,수영동강수,수영동기온
0,2020-01-01 00:00:00,0.1,-3.9,0.0,2.5
1,2020-01-01 01:00:00,0.0,-3.1,0.0,3.4
2,2020-01-01 02:00:00,0.0,-1.8,0.0,3.8
3,2020-01-01 03:00:00,0.0,-0.8,0.0,4.6
4,2020-01-01 04:00:00,0.0,-0.1,0.0,5.1


In [51]:
df['time']= pd.to_datetime(df['time'])
target = df[df['time'].dt.month.isin([6,7,8])]
ans = target[target['이화동기온'] > target['수영동기온']]
print(len(ans))
result = ans['time'].dt.hour
print(len(result))

1415
1415


In [69]:
df['time']= pd.to_datetime(df['time'])
ans1 = df[df['이화동강수']== df['이화동강수'].max()].time.values
print(ans1)
ans2 = df[df['수영동강수']== df['수영동강수'].max()].time.values
print(ans1, ans2)

['2020-09-30T09:00:00.000000000']
['2020-09-30T09:00:00.000000000'] ['2020-07-23T12:00:00.000000000']


In [1]:
import pandas as pd
#데이터 로드
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,6842,15793491,Cherkasova,714,Germany,Male,26,3,119545.48,2,1,0,65482.94,0
1,8963,15607874,Keane,687,France,Male,38,0,144450.58,1,0,1,137276.83,0
2,7047,15737627,Rivero,589,Germany,Female,20,2,121093.29,2,1,0,3529.72,0
3,7503,15697844,Whitehouse,721,Spain,Female,32,10,0.00,1,1,0,136119.96,1
4,3439,15722404,Carpenter,445,France,Female,30,3,0.00,2,1,1,127939.19,0


In [12]:
target = df[(df['Gender'] == 'Male') & (df['Exited']==1)]
ans = target.groupby('Geography')['Exited'].count().sort_values(ascending=False)
print(ans.head(1))

Geography
Germany    287
Name: Exited, dtype: int64


In [22]:
ans = df[(df['HasCrCard']== 1) & (df['IsActiveMember'] ==1)]['Age'].mean()
print(round(ans,4))


39.6102


In [26]:
ans = df[df['Balance'] >= df['Balance'].median()]['CreditScore'].std()
print(round(ans,3))

97.295


In [33]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")
df.head()
df.columns

Index(['성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg단위)', '허리둘레', '시력(좌)',
       '시력(우)', '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '총콜레스테롤',
       '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백', '혈청크레아티닌',
       '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '구강검진수검여부', '치아우식증유무',
       '치석'],
      dtype='object')

In [41]:
df['혈압차'] = df['수축기혈압'] - df['이완기혈압']
ans = df.groupby('연령대코드(5세단위)')['혈압차'].var().sort_values(ascending=False)
print(ans.index[4])

60


In [61]:
df['WHtR'] = df['허리둘레'] / df['신장(5Cm단위)']
target = df[df['WHtR'] >= 0.58]
ans = len(target[target['성별코드'] == 'M']) / len(target[target['성별코드'] == 'F'])
print(ans)

1.1693877551020408


In [62]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv")
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,188957,Female,25,1,17.0,1,< 1 Year,No,38785.0,152.0,59,0
1,275631,Male,20,1,22.0,0,< 1 Year,No,2630.0,160.0,113,0
2,329036,Female,40,1,41.0,0,1-2 Year,Yes,33165.0,124.0,188,0
3,227288,Female,28,1,30.0,1,< 1 Year,No,29038.0,152.0,233,0
4,300441,Female,50,1,28.0,0,1-2 Year,Yes,31325.0,124.0,258,0


In [70]:
target = df[df['Vehicle_Age'] == '> 2 Years']
ans = target[target['Annual_Premium'] >= target['Annual_Premium'].median()]['Vintage'].mean()
print(ans)

answer = df[(df['Vehicle_Age']=='> 2 Years') & (df['Annual_Premium'] >= df['Annual_Premium'].median())]['Vintage'].mean()
print(answer)

154.55906743858552
154.43647182359118


In [78]:
ans = df.groupby(['Vehicle_Age', 'Gender'],as_index=False)['Annual_Premium'].mean()
print(ans)
print()
result = ans.pivot(index='Vehicle_Age', columns='Gender',values='Annual_Premium')
print(result)

  Vehicle_Age  Gender  Annual_Premium
0    1-2 Year  Female    30762.245001
1    1-2 Year    Male    30413.088469
2    < 1 Year  Female    29972.286702
3    < 1 Year    Male    30310.982212
4   > 2 Years  Female    36108.366374
5   > 2 Years    Male    35303.870627

Gender             Female          Male
Vehicle_Age                            
1-2 Year     30762.245001  30413.088469
< 1 Year     29972.286702  30310.982212
> 2 Years    36108.366374  35303.870627


In [79]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [98]:
ans = df.groupby(['price_range','n_cores']).size().sort_values(ascending=False).groupby('price_range').head(1)
print(ans)


price_range  n_cores
1            4          76
2            4          73
3            5          70
0            2          69
dtype: int64


In [133]:
ans = df[df['price_range']==3].corr().unstack().dropna().sort_values(ascending=False)
ans = ans[ans.values != 1].iloc[1:2]
print(ans)

pc  fc    0.635166
dtype: float64


In [115]:
cordf = df.loc[df['price_range']==3].corr().unstack().sort_values(ascending=False)
answer  = cordf.loc[cordf!=1].reset_index().iloc[1]
print(answer)

level_0          pc
level_1          fc
0          0.635166
Name: 1, dtype: object
